In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

print("CHECK ALL FILES - COLUMNS + FIRST 3 ROWS")

files = {
    'GDC Sample Sheet': '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/gdc_sample_sheet.tsv',
    'Sample TSV': '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/sample.tsv',
    'Clinical TSV': '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/clinical.tsv',
    'Follow-up TSV': '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/follow_up.tsv',
    'Latent Features': '/content/drive/MyDrive/GDC TCGA DATASET/AutoEncoder/150_Features/merged_latent_150f.csv'
}

for name, path in files.items():
    print(f"FILE: {name}")
    print(f"Path: {path}")
    if path.endswith('.tsv'):
        df = pd.read_csv(path, sep='\t', low_memory=False)
    else:
        df = pd.read_csv(path)

    print(f"\nShape: {df.shape}")
    print(f"\nALL COLUMNS ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:3}. {col}")

    print(f"\nFIRST 3 ROWS:")
    print(df.head(3).to_string())

    print(f"\nKEY COLUMN VALUES (unique counts):")
    for col in df.columns[:10]:
        unique = df[col].nunique()
        print(f"  {col:50}: {unique:6} unique values")

print("DONE!")


CHECK ALL FILES - COLUMNS + FIRST 3 ROWS
FILE: GDC Sample Sheet
Path: /content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/gdc_sample_sheet.tsv

Shape: (7465, 11)

ALL COLUMNS (11):
    1. File ID
    2. File Name
    3. Data Category
    4. Data Type
    5. Project ID
    6. Case ID
    7. Sample ID
    8. Tissue Type
    9. Tumor Descriptor
   10. Specimen Type
   11. Preservation Method

FIRST 3 ROWS:
                                File ID                                                                      File Name    Data Category               Data Type Project ID            Case ID              Sample ID Tissue Type Tumor Descriptor Specimen Type Preservation Method
0  1b9881e3-ceac-4ecc-8387-bfcdbcbd3c52  6bd14673-1300-4f6c-b38a-94045a5e4267.methylation_array.sesame.level3betas.txt  DNA Methylation  Methylation Beta Value  HCMI-CMDC  HCM-STAN-0848-C20  HCM-STAN-0848-C20-01A       Tumor          Primary  Solid Tissue              Frozen
1  a211543c-825b-448e-adc4

In [3]:
import pandas as pd
import os
print("CORRECT MERGE - USING UUID CASE IDs")

gdc = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/gdc_sample_sheet.tsv', sep='\t')
gdc['methylation_sample_id'] = 'sample' + (gdc.index + 1).astype(str)

sample = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/sample.tsv', sep='\t')

merged = gdc.merge(sample, left_on='Sample ID', right_on='samples.submitter_id', how='left')

print(f"After sample merge: {merged.shape}")
print(f"Got UUID case_id: {merged['cases.case_id'].notna().sum()}")

clinical = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /TCGA_Clinical/clinical.tsv', sep='\t', low_memory=False)
clinical_agg = clinical.groupby('cases.case_id', as_index=False).first()

merged = merged.merge(clinical_agg, on='cases.case_id', how='left', suffixes=('', '_clin'))

print(f"After clinical merge: {merged.shape}")
print(f"Got vital_status: {merged['demographic.vital_status'].notna().sum()}")
print(f"Got stage: {merged['diagnoses.ajcc_pathologic_stage'].notna().sum()}")

latent = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/AutoEncoder/150_Features/merged_latent_150f.csv')
merged = merged.merge(latent, left_on='methylation_sample_id', right_on='sample_id', how='left')

print(f"After latent merge: {merged.shape}")
print(f"Got latent: {merged['latent_1'].notna().sum()}")

merged.to_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CORRECT.csv', index=False)

print(f"\n✓ SAVED! Check vital_status, stage, age!")


CORRECT MERGE - USING UUID CASE IDs
After sample merge: (7465, 51)
Got UUID case_id: 7464


/tmp/ipython-input-1855284465.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clinical_agg = clinical.groupby('cases.case_id', as_index=False).first()


After clinical merge: (7465, 260)
Got vital_status: 7464
Got stage: 7464
After latent merge: (7465, 411)
Got latent: 7404

✓ SAVED! Check vital_status, stage, age!


In [5]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CORRECT.csv')

print("MASTER METADATA - COMPLETE CHECK")

print(f"\nShape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")


print(f"ALL {df.shape[1]} COLUMNS:")

for i, col in enumerate(df.columns, 1):
    missing = df[col].isnull().sum()
    pct = (missing / len(df)) * 100
    print(f"{i:3}. {col:50} | Missing: {missing:5} ({pct:5.1f}%)")
print("FIRST 5 ROWS - KEY COLUMNS")

key_cols = [
    'methylation_sample_id',
    'Sample ID',
    'Case ID',
    'Tissue Type',
    'Project ID',
    'demographic.vital_status',
    'demographic.age_at_index',
    'demographic.gender',
    'diagnoses.ajcc_pathologic_stage',
    'diagnoses.primary_diagnosis',
    'latent_1',
    'latent_2',
    'latent_150'
]

available_cols = [c for c in key_cols if c in df.columns]
print(df[available_cols].head())

print("DATA SUMMARY")

if 'Tissue Type' in df.columns:
    print(f"\nTissue Type:")
    print(df['Tissue Type'].value_counts())

if 'demographic.vital_status' in df.columns:
    print(f"\nVital Status:")
    print(df['demographic.vital_status'].value_counts())

if 'diagnoses.ajcc_pathologic_stage' in df.columns:
    print(f"\nStage (top 10):")
    print(df['diagnoses.ajcc_pathologic_stage'].value_counts().head(10))

if 'demographic.gender' in df.columns:
    print(f"\nGender:")
    print(df['demographic.gender'].value_counts())

if 'Project ID' in df.columns:
    print(f"\nProjects: {df['Project ID'].nunique()}")

print(f"\nLatent features available: {df['latent_1'].notna().sum()}")

print(" DATA QUALITY CHECK")

checks = {
    'Total Samples': len(df),
    'Normal Tissue': (df['Tissue Type'] == 'Normal').sum() if 'Tissue Type' in df.columns else 'N/A',
    'Tumor Tissue': (df['Tissue Type'] == 'Tumor').sum() if 'Tissue Type' in df.columns else 'N/A',
    'With Vital Status': df['demographic.vital_status'].notna().sum() if 'demographic.vital_status' in df.columns else 'N/A',
    'With Stage': df['diagnoses.ajcc_pathologic_stage'].notna().sum() if 'diagnoses.ajcc_pathologic_stage' in df.columns else 'N/A',
    'With Latent Features': df['latent_1'].notna().sum() if 'latent_1' in df.columns else 'N/A',
    'Cancer Types': df['Project ID'].nunique() if 'Project ID' in df.columns else 'N/A'
}

for key, value in checks.items():
    print(f"  {key:25}: {value}")
print("MASTER METADATA READY FOR 4 PAPERS! ")


/tmp/ipython-input-4259722960.py:3: DtypeWarning: Columns (29,61,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CORRECT.csv')


MASTER METADATA - COMPLETE CHECK

Shape: (7465, 411)
Rows: 7,465
Columns: 411
ALL 411 COLUMNS:
  1. File ID                                            | Missing:     0 (  0.0%)
  2. File Name                                          | Missing:     0 (  0.0%)
  3. Data Category                                      | Missing:     0 (  0.0%)
  4. Data Type                                          | Missing:     0 (  0.0%)
  5. Project ID                                         | Missing:     0 (  0.0%)
  6. Case ID                                            | Missing:     0 (  0.0%)
  7. Sample ID                                          | Missing:     0 (  0.0%)
  8. Tissue Type                                        | Missing:     0 (  0.0%)
  9. Tumor Descriptor                                   | Missing:     0 (  0.0%)
 10. Specimen Type                                      | Missing:     0 (  0.0%)
 11. Preservation Method                                | Missing:     0 (  0.0%)
 12

In [6]:
import pandas as pd

print("REMOVE PATIENTS WITH UNKNOWN VITAL STATUS")

df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CORRECT.csv')

print(f"\nBEFORE CLEANING:")
print(f"  Total samples: {len(df):,}")

print(f"\nVital Status Distribution:")
print(df['demographic.vital_status'].value_counts())
clean_df = df[df['demographic.vital_status'].isin(['Alive', 'Dead'])].copy()

print(f"\nAFTER CLEANING:")
print(f"  Total samples: {len(clean_df):,}")
print(f"  Removed: {len(df) - len(clean_df)} patients")

print(f"\nVital Status Distribution (cleaned):")
print(clean_df['demographic.vital_status'].value_counts())

print(f"\nKEY FEATURES CHECK:")
print(f"  Tissue Type: {clean_df['Tissue Type'].notna().sum():,} / {len(clean_df):,}")
print(f"  Latent Features: {clean_df['latent_1'].notna().sum():,} / {len(clean_df):,}")
print(f"  Stage: {clean_df['diagnoses.ajcc_pathologic_stage'].notna().sum():,} / {len(clean_df):,}")
print(f"  Gender: {clean_df['demographic.gender'].notna().sum():,} / {len(clean_df):,}")

output_file = '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CLEANED.csv'
clean_df.to_csv(output_file, index=False)

print(f" SAVED CLEANED FILE")
print(f"  Location: {output_file}")
print(f"  Final samples: {len(clean_df):,}")
print(f"  Columns: {clean_df.shape[1]}")

print(f"\nFINAL DATA SUMMARY:")
print(f"  Alive: {(clean_df['demographic.vital_status'] == 'Alive').sum():,}")
print(f"  Dead: {(clean_df['demographic.vital_status'] == 'Dead').sum():,}")
print(f"  Normal tissue: {(clean_df['Tissue Type'] == 'Normal').sum():,}")
print(f"  Tumor tissue: {(clean_df['Tissue Type'] == 'Tumor').sum():,}")

print(f"\n CLEANED MASTER READY FOR ALL 4 PAPERS! ")


REMOVE PATIENTS WITH UNKNOWN VITAL STATUS


/tmp/ipython-input-197253899.py:5: DtypeWarning: Columns (29,61,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CORRECT.csv')



BEFORE CLEANING:
  Total samples: 7,465

Vital Status Distribution:
demographic.vital_status
Alive           5306
Dead            2106
'--               26
Not Reported      24
Unknown            2
Name: count, dtype: int64

AFTER CLEANING:
  Total samples: 7,412
  Removed: 53 patients

Vital Status Distribution (cleaned):
demographic.vital_status
Alive    5306
Dead     2106
Name: count, dtype: int64

KEY FEATURES CHECK:
  Tissue Type: 7,412 / 7,412
  Latent Features: 7,404 / 7,412
  Stage: 7,412 / 7,412
  Gender: 7,412 / 7,412
 SAVED CLEANED FILE
  Location: /content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CLEANED.csv
  Final samples: 7,412
  Columns: 411

FINAL DATA SUMMARY:
  Alive: 5,306
  Dead: 2,106
  Normal tissue: 777
  Tumor tissue: 6,635

 CLEANED MASTER READY FOR ALL 4 PAPERS! 


In [7]:
import pandas as pd
import numpy as np

print("PROPER MISSING VALUES CHECK (INCLUDING '--' and '__')")

df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CLEANED.csv', low_memory=False)

print(f"\nTotal Samples: {len(df):,}")
print(f"Total Columns: {df.shape[1]}")
def is_missing(value):
    if pd.isna(value):
        return True
    if isinstance(value, str):
        value_lower = value.lower().strip()
        missing_values = ["'--", '--', '__', 'not reported', 'unknown', 'not available', 'na', 'n/a']
        return value_lower in missing_values
    return False

missing_report = []

for col in df.columns:
    nan_count = df[col].isnull().sum()

    placeholder_count = df[col].apply(is_missing).sum()

    total_missing = placeholder_count
    missing_pct = (total_missing / len(df)) * 100
    available = len(df) - total_missing
    available_pct = 100 - missing_pct

    missing_report.append({
        'Column': col,
        'NaN_Count': nan_count,
        'Placeholder_Count': placeholder_count - nan_count,
        'Total_Missing': total_missing,
        'Missing_Percent': missing_pct,
        'Available': available,
        'Available_Percent': available_pct
    })

report_df = pd.DataFrame(missing_report)
report_df = report_df.sort_values('Missing_Percent', ascending=False)
print("ALL 411 COLUMNS - TRUE MISSING VALUES")

for idx, row in report_df.iterrows():
    print(f"{row['Column']:50} | NaN: {row['NaN_Count']:5.0f} | Placeholder: {row['Placeholder_Count']:5.0f} | Total Missing: {row['Total_Missing']:5.0f} ({row['Missing_Percent']:5.1f}%) | Available: {row['Available']:5.0f} ({row['Available_Percent']:5.1f}%)")

print("SUMMARY")

print(f"\nColumns by Completeness:")
print(f"  100% complete (0% missing):     {(report_df['Missing_Percent'] == 0).sum()} columns")
print(f"  >95% complete (<5% missing):    {(report_df['Missing_Percent'] < 5).sum()} columns")
print(f"  >90% complete (<10% missing):   {(report_df['Missing_Percent'] < 10).sum()} columns")
print(f"  >80% complete (<20% missing):   {(report_df['Missing_Percent'] < 20).sum()} columns")
print(f"  >50% complete (<50% missing):   {(report_df['Missing_Percent'] < 50).sum()} columns")
print(f"  <50% complete (>50% missing):   {(report_df['Missing_Percent'] >= 50).sum()} columns")
print(f"  100% missing:                   {(report_df['Missing_Percent'] == 100).sum()} columns")

output_file = '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MISSING_VALUES_TRUE_REPORT.csv'
report_df.to_csv(output_file, index=False)

print(f"\n Saved: {output_file}")

print("TOP 30 COLUMNS WITH MOST MISSING DATA")
print(report_df[['Column', 'Total_Missing', 'Missing_Percent']].head(30).to_string(index=False))

complete = report_df[report_df['Missing_Percent'] == 0]
print(f"COLUMNS WITH 0% MISSING ({len(complete)} columns)")
for col in complete['Column'].tolist():
    print(f"  {col}")

print("PROPER MISSING VALUES CHECK COMPLETE!")


PROPER MISSING VALUES CHECK (INCLUDING '--' and '__')

Total Samples: 7,412
Total Columns: 411
ALL 411 COLUMNS - TRUE MISSING VALUES
treatments.treatment_outcome_duration              | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0%)
samples.growth_rate                                | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0%)
samples.distributor_reference                      | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0%)
samples.distance_normal_to_tumor                   | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0%)
samples.current_weight                             | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0%)
samples.catalog_reference                          | NaN:     0 | Placeholder:  7412 | Total Missing:  7412 (100.0%) | Available:     0 (  0.0

In [8]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CLEANED.csv', low_memory=False)

print("FIRST 5 SAMPLES - ALL 411 COLUMNS")

for sample_num in range(5):
    print(f"SAMPLE {sample_num + 1}")

    row = df.iloc[sample_num]

    for col_name, value in row.items():
        print(f"{col_name:50} = {value}")

print("DONE!")


FIRST 5 SAMPLES - ALL 411 COLUMNS
SAMPLE 1
File ID                                            = 1b9881e3-ceac-4ecc-8387-bfcdbcbd3c52
File Name                                          = 6bd14673-1300-4f6c-b38a-94045a5e4267.methylation_array.sesame.level3betas.txt
Data Category                                      = DNA Methylation
Data Type                                          = Methylation Beta Value
Project ID                                         = HCMI-CMDC
Case ID                                            = HCM-STAN-0848-C20
Sample ID                                          = HCM-STAN-0848-C20-01A
Tissue Type                                        = Tumor
Tumor Descriptor                                   = Primary
Specimen Type                                      = Solid Tissue
Preservation Method                                = Frozen
methylation_sample_id                              = sample1
project.project_id                                 = HCMI-CMDC
cases.case_i

In [10]:
import pandas as pd

print("CREATE FINAL PREPROCESSED DATA FOR 4 EXPERIMENTS")

df = pd.read_csv('/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/MASTER_CLEANED.csv', low_memory=False)

print(f"\nBEFORE PREPROCESSING:")
print(f"  Total samples: {len(df):,}")
print(f"  Total columns: {df.shape[1]}")


keep_columns = [
    'methylation_sample_id',
    'File ID',
    'Case ID',
    'Sample ID',

    'Project ID',
    'cases.disease_type',

    'Tissue Type',
    'samples.tissue_type',
    'samples.sample_type',

    'demographic.vital_status',
    'demographic.gender',
    'demographic.days_to_death',
    'demographic.ethnicity',
    'demographic.race',

    'diagnoses.primary_diagnosis',
    'diagnoses.ajcc_pathologic_stage',
    'diagnoses.tumor_grade',
    'diagnoses.age_at_diagnosis',
    'diagnoses.days_to_diagnosis',
    'diagnoses.morphology',
    'diagnoses.tissue_or_organ_of_origin',
    'diagnoses.metastasis_at_diagnosis',
    'diagnoses.progression_or_recurrence',
    'diagnoses.prior_malignancy',
    'diagnoses.prior_treatment',

    'follow_ups.disease_response',
    'follow_ups.days_to_follow_up',
    'follow_ups.progression_or_recurrence',
    'follow_ups.days_to_progression'
]

latent_columns = [f'latent_{i}' for i in range(1, 151)]
keep_columns.extend(latent_columns)

available_columns = [col for col in keep_columns if col in df.columns]

print(f"\nSTEP 1: SELECT ESSENTIAL COLUMNS")
print(f"  Requested: {len(keep_columns)} columns")
print(f"  Available: {len(available_columns)} columns")

final_df = df[available_columns].copy()


print(f"\nSTEP 2: REMOVE ROWS WITHOUT LATENT FEATURES")
print(f"  Before: {len(final_df):,} samples")

final_df = final_df[final_df['latent_1'].notna()].copy()

print(f"  After: {len(final_df):,} samples")
print(f"  Removed: {len(df) - len(final_df)} samples")


print("FINAL DATA SUMMARY")

print(f"\nShape: {final_df.shape}")
print(f"  Rows (samples): {len(final_df):,}")
print(f"  Columns: {final_df.shape[1]}")

print(f"\nTissue Type Distribution:")
if 'Tissue Type' in final_df.columns:
    print(final_df['Tissue Type'].value_counts())

print(f"\nVital Status Distribution:")
if 'demographic.vital_status' in final_df.columns:
    print(final_df['demographic.vital_status'].value_counts())

print(f"\nGender Distribution:")
if 'demographic.gender' in final_df.columns:
    print(final_df['demographic.gender'].value_counts())



output_file = '/content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/Final_Preprocess_Data_for_4_Experiments.csv'
final_df.to_csv(output_file, index=False)

print(" FINAL FILE SAVED!")

print(f"\nFile: {output_file}")
print(f"Size: {final_df.shape}")



CREATE FINAL PREPROCESSED DATA FOR 4 EXPERIMENTS

BEFORE PREPROCESSING:
  Total samples: 7,412
  Total columns: 411

STEP 1: SELECT ESSENTIAL COLUMNS
  Requested: 179 columns
  Available: 175 columns

STEP 2: REMOVE ROWS WITHOUT LATENT FEATURES
  Before: 7,412 samples
  After: 7,404 samples
  Removed: 8 samples
FINAL DATA SUMMARY

Shape: (7404, 175)
  Rows (samples): 7,404
  Columns: 175

Tissue Type Distribution:
Tissue Type
Tumor     6628
Normal     776
Name: count, dtype: int64

Vital Status Distribution:
demographic.vital_status
Alive    5299
Dead     2105
Name: count, dtype: int64

Gender Distribution:
demographic.gender
male      3732
female    3672
Name: count, dtype: int64
 FINAL FILE SAVED!

File: /content/drive/MyDrive/GDC TCGA DATASET/1. Capstone /processed_data/Final_Preprocess_Data_for_4_Experiments.csv
Size: (7404, 175)
